# Cleaning and merging weather and power data
This file contains general purpose scripts for downloading, cleaning, and merging weather and power data.


In [1]:
import reverse_geocoder as rg
import pandas as pd
import numpy as np
import ftplib
import requests
import matplotlib.pyplot as plt
import os
import re

pd.options.mode.copy_on_write = True

## Automatic downloading and reading of files

In [2]:
event_types = {'hail':'hail', 'storm_structure':'structure', 'tornados':'tvs', 'lightning':'nldn-tiles', 'mesocyclone':'mda'}
for event in event_types:
    path = '../../weather_data/'+event
    if not os.path.exists(path):
        os.mkdir(path)

In [3]:
# connect to the server
ftp = ftplib.FTP('ftp.ncdc.noaa.gov', timeout=30) #pass the url without protocol
ftp.login() #pass credentials if anonymous access is not allowed

# switch to the directory containing the data
ftp.cwd('/pub/data/swdi/database-csv/v2/')
ftp.pwd()

httpurl = 'https://www.ncei.noaa.gov/pub/data/swdi/database-csv/v2/'
# get the list of files in this ftp dir
all_files= ftp.nlst()

In [4]:
def download_weather(year, event_type):
    print(f'Downloading {event_type} file for {year}.')
    event_name = event_types[event_type]
    pattern = event_name+"-"+str(year)
    file_name = [fname for fname in all_files if pattern in fname]
    if len(file_name) == 0:
        print("No file in that year for that event type" )
        return 
    file_name = file_name[0]
    print("Considering file ", file_name)
    if os.path.exists('../../weather_data/{}/{}'.format(event_type, file_name)):
        print("file already exists")
        return 
    query_parameters = {"downloadformat": "csv"}
    print("Getting the response from the URL .....")
    response = requests.get(httpurl+file_name, params=query_parameters)
    if response.ok:
        print("Downloaded succesfully")
    with open(r'../../weather_data/{}/{}'.format(event_type, file_name), "wb") as f:
        f.write(response.content)
    print('Saved in folder')

In [5]:
def read_weather(year, event_type):
    download_weather(year,event_type)
    files = os.listdir('../../weather_data/'+event_type)
    file_name = [fname for fname in files if str(year) in fname]
    if len(file_name) == 0:
        raise Exception(f"No file for event type {event_type} in year {year}") 
    if len(file_name) > 1:
        raise Exception(f"Multiple files for event type {event_type} in year {year}")
    if event_type == 'lightning' or event_type == 'tornado':
        return pd.read_csv(r'../../weather_data/'+event_type+'/'
                  + file_name[0], skiprows=2, parse_dates=['#ZDAY'])
    return pd.read_csv(r'../../weather_data/'+event_type+'/'
                  + file_name[0], skiprows=2, parse_dates=['#ZTIME'])

## Cleaning power data
We clean the power data as follows.
1. Convert 'Date Event Began' column to datetime format.
2. Keep only the rows where power event type involves 'Severe Weather'.
3. Drop all columns except Date Event Began and Area Affected

In [6]:
def clean_power(power):
    power['Date Event Began'] = pd.to_datetime(power['Date Event Began'], format='%m/%d/%Y')
    power = power[power['Event Type'].str.contains(r'Severe Weather', regex=True)]
    return power.drop(columns=['Month', 'Time Event Began', 'Date of Restoration', 'Time of Restoration', 
                         'NERC Region', 'Alert Criteria', 'Event Type', 'Demand Loss (MW)', 'Number of Customers Affected'])

## Cleaning weather data
We clean weather data as follows.
1. Convert '#ZDAY' or '#ZTIME' column to 'DATE' column in datetime format, and add a separate 'MONTH' column as well.
2. Split into groups by 'DATE', 'WSR_ID', and 'CELL_ID', then get the mean of 'LAT' and 'LON' in each group, and the max for each remaining attribute in each group. (For lightning data, this step is not necessary.)
3. Reverse geosearch using 'LAT' and 'LON' to get 'COUNTY' and 'STATE' columns.
4. Drop the rows where no county info is found (indicating an event outside of the US).

In [7]:
def get_date(weather):
    weather['DATE'] = pd.to_datetime(weather['#ZTIME']).dt.normalize()
    weather['MONTH'] = weather['DATE'].dt.month
    return weather.drop(columns=['#ZTIME'])

In [8]:
def aggregate_groups(weather, attributes):
    groups = weather.groupby(['DATE', 'MONTH', 'WSR_ID', 'CELL_ID'], as_index=False)
    return groups.agg(attributes)

In [9]:
def get_county_and_state(weather):
    addresses = rg.search(list(zip(weather['LAT'], weather['LON'])))
    weather['COUNTY'] = [x['admin2'] for x in addresses]
    weather['STATE'] = [x['admin1'] for x in addresses]
    return weather[weather['COUNTY'] != '']

In [10]:
def clean_lightning(lightning):
    lightning.rename(columns={'#ZDAY' : '#ZTIME', 'CENTERLAT': 'LAT', 'CENTERLON': 'LON'}, inplace=True)
    lightning_dates = get_date(lightning)
    return get_county_and_state(lightning_dates)

In [11]:
def clean_tornado(tornados):
    tornados_dates = get_date(tornados)
    tornados_aggregate = aggregate_groups(tornados_dates, {'LAT':'mean',
                                                           'LON':'mean',
                                                           'AVGDV':'max', 'LLDV':'max', 'MXDV':'max',
                                                           'MXDV_HEIGHT':'max', 'DEPTH':'max',
                                                           'MAX_SHEAR':'max', 'MAX_SHEAR_HEIGHT':'max'})
    tor_agg = get_county_and_state(tornados_aggregate)
    return tor_agg

In [12]:
def clean_hail(hail):
    hail = hail[hail.SEVPROB>0]
    hail_dates = get_date(hail)
    hail_aggregate = aggregate_groups(hail_dates, {'LAT':'mean', 
                                                   'LON':'mean',
                                                   'SEVPROB':'max', 'PROB':'max', 'MAXSIZE':'max'})
    hail_agg = get_county_and_state(hail_aggregate)
    return hail_agg

In [13]:
def clean_meso(meso):
    meso_dates = get_date(meso)
    meso_aggregate = aggregate_groups(meso_dates, {'LAT':'mean', 
                                                  'LON':'mean', 
                                                  'STR_RANK':'max', 'LL_ROT_VEL':'max', 
                                                  'LL_DV':'max', 'LL_BASE':'max', 'DEPTH_KFT':'max', 
                                                  'DPTH_STMRL':'max', 'MAX_RV_KFT':'max', 'MAX_RV_KTS':'max', 
                                                  'TVS':'max', 'MSI':'max'})
    meso_agg = get_county_and_state(meso_aggregate)
    return meso_agg

In [14]:
def clean_storm(storm):
    storm_dates = get_date(storm)
    storm_aggregate = aggregate_groups(storm_dates, {'LAT':'mean',
                                                     'LON':'mean',
                                                     'MAX_REFLECT':'max', 'VIL':'max', 'HEIGHT':'max'})
    storm_agg = get_county_and_state(storm_aggregate)
    return storm_agg

## Merging
We merge cleaned weather data with cleaned power data by adding a column specifying whether or not the recorded weather event resulted in a power outage in the same area on the same date. 'In the same area' is explained in the in_area function.

### Checking if weather and power event are in same county, or state if no county info

In [15]:
# get all US states
counties = pd.read_csv("../../county_data/uscounties.csv", index_col=0)
counties['county'] = counties['county'].astype(str)

In [16]:
def in_area(county, state, area_affected):
    """
    input:

    county, state: the county and state of the weather event
    area_affected: the area affected by the power outage(a string listing states and possibly counties)
    has_county_info: whether area_affected

    output: True if either state and county are both in area_affected, or
            False if state is in area_affected and there is no county info for area_affected
    """
    if not county or not state or not area_affected:
        raise Exception(f"Invalid (null) input. county: {county}, state: {state}, area_affected: {area_affected}")

    # adding a colon to state ensures that it's matched exactly to a state in area_affected
    # (rather than a county whose name is a state)
    stateC = ''.join([state,':'])

    # has_county_info is True if area_affected includes a county, false otherwise
    has_county_info = any(cty in area_affected for cty in counties[counties['state'] == state]['county'])
    
    return stateC in area_affected and (county in area_affected or not has_county_info)

### Merging weather and power

In [17]:
def merge_weather_power(weather, power, light=True):
    """
    Merge weather and power data.

    Assumes that the input data are already cleaned.
    """
    merged = pd.merge(weather, power, how='left', left_on='DATE', right_on='Date Event Began', indicator=True)
    merged['POWER_OUTAGE'] = merged.apply(lambda row: (row['_merge'] == 'both') and in_area(str(row['COUNTY']),
                                                                                            str(row['STATE']),
                                                                                            str(row['Area Affected'])),
                                          axis = 'columns')
    if light:
        try:
            return merged.drop(columns=['Date Event Began', 'Area Affected', '_merge', 'WSR_ID', 'CELL_ID'])
        except KeyError:
            return merged.drop(columns=['Date Event Began', 'Area Affected', '_merge'])
    return merged.drop(columns=['Date Event Began', 'Area Affected', '_merge'])

In [18]:
def merge(year, event_type):
    weather = read_weather(year,event_type)
    if event_type == 'hail':
        weather = clean_hail(weather)
    elif event_type == 'storm_structure':
        weather = clean_storm(weather)
    elif event_type == 'tornados':
        weather = clean_tornado(weather)
    elif event_type == 'lightning':
        weather = clean_lightning(weather)
    elif event_type == 'mesocyclone':
        weather = clean_meso(weather)
    else:
        raise Exception(f'Invalid event type: {event_type}, must be one of {event_types.keys()}')
    power= pd.read_excel('../../power_data/' + str(year) + '_Annual_Summary.xls', skiprows=1)
    power = clean_power(power)
    return merge_weather_power(weather,power)

In [20]:
merged_meso = merge(2019,'mesocyclone')
merged_meso.to_csv(f'../../merged/merged_meso_2019.csv.gz', compression='gzip')

Considering file  mda-2019.csv.gz
file already exists
Loading formatted geocoded file...
WARNING *** file size (96452) not 512 + multiple of sector size (512)


In [10]:
%%time
for year in range(2015,2023):
    merged_hail = merge(year,'hail')
    merged_hail.to_csv(f'../../merged/merged_hail_{year}.csv.gz', compression='gzip')

NameError: name 'read_weather' is not defined

In [22]:
%%time
merged_storm_2020 = merge(2020,'storm_structure')

Considering file  structure-2020.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (120004) not 512 + multiple of sector size (512)
CPU times: user 9min 25s, sys: 8.51 s, total: 9min 34s
Wall time: 10min 17s


In [23]:
merged_storm_2020.head()

,DATE,MONTH,LAT,LON,MAX_REFLECT,VIL,HEIGHT,COUNTY,STATE,POWER_OUTAGE
0,2020-01-01,1,45.531415,-98.985618,38,1,3.2,Edmunds County,South Dakota,False
1,2020-01-01,1,45.057635,-98.531035,38,0,2.9,Spink County,South Dakota,False
2,2020-01-01,1,45.122425,-97.951135,40,1,2.1,Clark County,South Dakota,False
3,2020-01-01,1,45.072180,-97.959420,35,0,2.2,Clark County,South Dakota,False
4,2020-01-01,1,24.273560,-80.413000,47,6,8.8,Monroe County,Florida,False


In [40]:
%%time
for year in range(2015,2023):
    merged_tornado = merge(year,'tornados')
    merged_tornado.to_csv(f'../../merged/merged_tornado_{year}.csv.gz', compression='gzip')

Considering file  tvs-2015.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  tvs-2016.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  tvs-2017.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (69316) not 512 + multiple of sector size (512)
Considering file  tvs-2018.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (86212) not 512 + multiple of sector size (512)
Considering file  tvs-2019.csv.gz
file already exists
WARNING *** file size (96452) not 512 + multiple of sector size (512)
Considering file  tvs-2020.csv.gz
file already exists
WARNING *** file size (120004) not 512 + multiple of sector size (512)
Considering file  t

In [38]:
%%time
for year in range(2015,2023):
    merged_meso = merge(year,'mesocyclone')
    merged_meso.to_csv(f'../../merged/merged_meso_{year}.csv.gz', compression='gzip')

Considering file  mda-2015.csv.gz
file already exists
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  mda-2016.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  mda-2017.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (69316) not 512 + multiple of sector size (512)
Considering file  mda-2018.csv.gz
Getting the response from the URL .....
Downloaded succesfully
Saved in folder
WARNING *** file size (86212) not 512 + multiple of sector size (512)
Considering file  mda-2019.csv.gz
file already exists
WARNING *** file size (96452) not 512 + multiple of sector size (512)
Considering file  mda-2020.csv.gz
file already exists
WARNING *** file size (120004) not 512 + multiple of sector size (512)
Considering file  mda-2021.csv.gz
Getting the response from the URL .....
Down

In [35]:
%%time
for year in range(2015,2023):
    merged_lightning = merge(year,'lightning')
    merged_lightning.to_csv(f'../../merged/merged_lightning_{year}.csv.gz', compression='gzip')

Considering file  nldn-tiles-2015.csv.gz
file already exists
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2016.csv.gz
file already exists
WARNING *** file size (73412) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2017.csv.gz
file already exists
WARNING *** file size (69316) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2018.csv.gz
file already exists
WARNING *** file size (86212) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2019.csv.gz
file already exists
WARNING *** file size (96452) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2020.csv.gz
file already exists
WARNING *** file size (120004) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2021.csv.gz
file already exists
WARNING *** file size (133828) not 512 + multiple of sector size (512)
Considering file  nldn-tiles-2022.csv.gz
file already exists
WARNING *** file siz